In [2]:
import pandas as pd
from sklearn.calibration import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_classif, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SequentialFeatureSelector

In [3]:
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

df1 = df1.drop(columns=['sub_area'])
df2_dropped = df2.drop(columns=['sub_area','row ID'])

In [4]:
categorical_columns = df1.select_dtypes(include=['object']).columns

#Label encode categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
  df1[col] = label_encoder.fit_transform(df1[col])
  df2_dropped[col] = label_encoder.transform(df2_dropped[col])


#df1_encoded = df1_encoded.drop(columns=['area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_education_centers_raion', 'children_school', 'school_education_centers_raion', 'healthcare_centers_raion', 'sport_objects_raion', 'additional_education_raion', 'shopping_centers_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_panel', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'ID_metro', 'metro_min_avto', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk', 'park_km', 'green_zone_km', 'water_treatment_km', 'cemetery_km', 'incineration_km', 'railroad_station_walk_km', 'railroad_station_walk_min', 'ID_railroad_station_walk', 'railroad_station_avto_km', 'railroad_station_avto_min', 'ID_railroad_station_avto', 'water_km', 'mkad_km', 'ttk_km', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'ID_big_road1', 'big_road2_km', 'ID_big_road2', 'railroad_km', 'zd_vokzaly_avto_km', 'ID_railroad_terminal', 'bus_terminal_avto_km', 'ID_bus_terminal', 'oil_chemistry_km', 'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km', 'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km', 'basketball_km', 'hospice_morgue_km', 'detention_facility_km', 'university_km', 'workplaces_km', 'shopping_centers_km', 'office_km', 'mosque_km', 'theater_km', 'museum_km', 'exhibition_km', 'catering_km', 'green_part_500', 'green_part_1000', 'prom_part_1000', 'green_part_1500', 'prom_part_1500', 'sport_count_1500', 'market_count_1500', 'green_part_2000', 'prom_part_2000', 'trc_count_2000', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'mosque_count_2000', 'sport_count_2000', 'market_count_2000', 'green_part_3000', 'prom_part_3000', 'office_sqm_3000', 'trc_count_3000', 'trc_sqm_3000', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'mosque_count_3000', 'sport_count_3000', 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'office_count_5000', 'office_sqm_5000', 'trc_count_5000', 'trc_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'mosque_count_5000', 'leisure_count_5000', 'sport_count_5000', 'market_count_5000', 'product_type_OwnerOccupier', 'culture_objects_top_25_yes', 'thermal_power_plant_raion_yes', 'incineration_raion_yes', 'oil_chemistry_raion_yes', 'radiation_raion_yes', 'railroad_terminal_raion_yes', 'big_market_raion_yes', 'nuclear_reactor_raion_yes', 'detention_facility_raion_yes', 'water_1line_yes', 'big_road1_1line_yes', 'railroad_1line_yes', 'ecology_good', 'ecology_no data', 'ecology_poor', 'ecology_satisfactory'],axis=1)
#df2_encoded = df2_encoded.drop(columns=['area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_education_centers_raion', 'children_school', 'school_education_centers_raion', 'healthcare_centers_raion', 'sport_objects_raion', 'additional_education_raion', 'shopping_centers_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female', '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male', '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all', '0_13_male', '0_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_panel', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_1921-1945', 'build_count_1946-1970', 'build_count_1971-1995', 'ID_metro', 'metro_min_avto', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk', 'park_km', 'green_zone_km', 'water_treatment_km', 'cemetery_km', 'incineration_km', 'railroad_station_walk_km', 'railroad_station_walk_min', 'ID_railroad_station_walk', 'railroad_station_avto_km', 'railroad_station_avto_min', 'ID_railroad_station_avto', 'water_km', 'mkad_km', 'ttk_km', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'ID_big_road1', 'big_road2_km', 'ID_big_road2', 'railroad_km', 'zd_vokzaly_avto_km', 'ID_railroad_terminal', 'bus_terminal_avto_km', 'ID_bus_terminal', 'oil_chemistry_km', 'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km', 'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km', 'basketball_km', 'hospice_morgue_km', 'detention_facility_km', 'university_km', 'workplaces_km', 'shopping_centers_km', 'office_km', 'mosque_km', 'theater_km', 'museum_km', 'exhibition_km', 'catering_km', 'green_part_500', 'green_part_1000', 'prom_part_1000', 'green_part_1500', 'prom_part_1500', 'sport_count_1500', 'market_count_1500', 'green_part_2000', 'prom_part_2000', 'trc_count_2000', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'mosque_count_2000', 'sport_count_2000', 'market_count_2000', 'green_part_3000', 'prom_part_3000', 'office_sqm_3000', 'trc_count_3000', 'trc_sqm_3000', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'mosque_count_3000', 'sport_count_3000', 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'office_count_5000', 'office_sqm_5000', 'trc_count_5000', 'trc_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'mosque_count_5000', 'leisure_count_5000', 'sport_count_5000', 'market_count_5000', 'product_type_OwnerOccupier', 'culture_objects_top_25_yes', 'thermal_power_plant_raion_yes', 'incineration_raion_yes', 'oil_chemistry_raion_yes', 'radiation_raion_yes', 'railroad_terminal_raion_yes', 'big_market_raion_yes', 'nuclear_reactor_raion_yes', 'detention_facility_raion_yes', 'water_1line_yes', 'big_road1_1line_yes', 'railroad_1line_yes', 'ecology_good', 'ecology_no data', 'ecology_poor', 'ecology_satisfactory'],axis=1)
print(df2_dropped.shape)

(77789, 272)


In [5]:
X = df1.drop(columns=['price_doc',],axis=1)
y = df1['price_doc']


In [7]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
df2_encoded = imputer.fit_transform(df2_dropped)

In [8]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
df2_encoded_scaled = scaler.fit_transform(df2_encoded)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=2)


In [10]:
variance_threshold = 0.02 # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_train_high_variance = selector.fit_transform(X_scaled)
X_test_high_variance = selector.transform(df2_encoded_scaled)

In [13]:
model = CatBoostRegressor(depth=5,iterations=350,l2_leaf_reg=2,learning_rate=0.08,random_strength=3, loss_function='RMSE', verbose=False)
model.fit(X_train_high_variance,y)

md_probs = model.predict(X_test_high_variance)



print(md_probs)

[12592729.26119724  6279054.07586412  4824155.35195291 ...
  2900570.06914754  2900570.06914754  2900570.06914754]


In [14]:
submission_df = pd.DataFrame({'row ID': df2['row ID'], 'price_doc': md_probs.flatten()})

# Save the results to a CSV file
submission_df.to_csv('predictions81.csv', index=False)